[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Use PolygonZone and Roboflow Supervision

In this notebook, you will use [PolygonZone](https://roboflow.github.io/polygonzone/) with [Roboflow Supervision](https://roboflow.com/supervision) to draw polygons on a video frame. These polygons will be used as zones in which predictions will be grouped.

This notebook accompanies the "Calculate Polygon Coordinates with PolygonZone" tutorial on the Roboflow blog.

## Pro Tip: Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

## Steps in this Tutorial

In this guide, we will:

1. Install supervision and YOLOv8.
2. Prepare polygon zones for a traffic video.
3. Run inference on a traffic video.
4. Save the results of inference to a file.

**Let's begin!**

## Install Dependencies and Retrieve Video

Install the required dependencies for this project. We'll be using Ultralytics' YOLOv8 model for inference, and Supervision for drawing our polygons and calculating how many objects appear in each annotated zone.

In [11]:
import subprocess
import sys
import os

# Check what package managers are available
print("Checking available package managers...")

# Try using uv which is available in the project
try:
    result = subprocess.run(["uv", "--version"], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ UV available: {result.stdout.strip()}")
        
        # Install packages using uv
        print("Installing packages with uv...")
        subprocess.check_call(["uv", "add", "ultralytics<=8.3.40", "supervision"])
        print("✅ Packages installed successfully with uv!")
        
except Exception as e:
    print(f"❌ UV installation failed: {e}")
    
    # Fallback: try direct installation without pip
    try:
        print("Trying alternative installation method...")
        import importlib.util
        
        # Check if packages are already available
        ultralytics_spec = importlib.util.find_spec("ultralytics")
        supervision_spec = importlib.util.find_spec("supervision")
        
        if ultralytics_spec and supervision_spec:
            print("✅ Packages already available!")
        else:
            print("❌ Packages not found. Manual installation required.")
            
    except Exception as e2:
        print(f"❌ Alternative method failed: {e2}")

# Fix matplotlib backend issue
os.environ['MPLBACKEND'] = 'Agg'

# Test imports
try:
    import ultralytics
    import supervision
    print("✅ All packages imported successfully!")
    
    # Configure ultralytics settings
    from ultralytics import settings
    settings.update({'sync': False})
    print("✅ Ultralytics configured successfully!")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please ensure packages are installed in the environment.")

Checking available package managers...
✅ UV available: uv 0.7.9
Installing packages with uv...


Resolved 88 packages in 163ms
Prepared 10 packages in 498ms
Prepared 10 packages in 498ms
Installed 44 packages in 145ms
 + certifi==2025.7.14
 + charset-normalizer==3.4.2
 + defusedxml==0.7.1
 + filelock==3.18.0
 + fsspec==2025.7.0
 + idna==3.10
 + jinja2==3.1.6
 + markupsafe==3.0.2
 + mpmath==1.3.0
 + networkx==3.5
 + nvidia-cublas-cu12==12.6.4.1
 + nvidia-cuda-cupti-cu12==12.6.80
 + nvidia-cuda-nvrtc-cu12==12.6.77
 + nvidia-cuda-runtime-cu12==12.6.77
 + nvidia-cudnn-cu12==9.5.1.17
 + nvidia-cufft-cu12==11.3.0.4
 + nvidia-cufile-cu12==1.11.1.6
 + nvidia-curand-cu12==10.3.7.77
 + nvidia-cusolver-cu12==11.7.1.2
 + nvidia-cusparse-cu12==12.5.4.2
 + nvidia-cusparselt-cu12==0.6.3
 + nvidia-nccl-cu12==2.26.2
 + nvidia-nvjitlink-cu12==12.6.85
 + nvidia-nvtx-cu12==12.6.77
 + opencv-python==4.11.0.86
 + pandas==2.3.1
 + py-cpuinfo==9.0.0
 + pytz==2025.2
 + pyyaml==6.0.2
 + requests==2.32.4
 + scipy==1.16.0
 + seaborn==0.13.2
 + setuptools==80.9.0
 + supervision==0.26.0
 + sympy==1.14.0
 + tor

✅ Packages installed successfully with uv!
✅ All packages imported successfully!
✅ Ultralytics configured successfully!
✅ All packages imported successfully!
✅ Ultralytics configured successfully!


In [15]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1K15ijbTl78VSOPjfvGSgvqh7ME2U7cG2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1K15ijbTl78VSOPjfvGSgvqh7ME2U7cG2" -O video.mp4 && rm -rf /tmp/cookies.txt

--2025-07-16 14:05:59--  https://docs.google.com/uc?export=download&confirm=&id=1K15ijbTl78VSOPjfvGSgvqh7ME2U7cG2
Resolving docs.google.com (docs.google.com)... 74.125.68.138, 74.125.68.102, 74.125.68.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.68.138|:443... connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1K15ijbTl78VSOPjfvGSgvqh7ME2U7cG2&export=download [following]
--2025-07-16 14:05:59--  https://drive.usercontent.google.com/download?id=1K15ijbTl78VSOPjfvGSgvqh7ME2U7cG2&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.68.132, 2404:6800:4003:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.68.132|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1K15ijbTl78VSOPjfvGSgvqh7ME2U7cG2&expor

## Initialize the Model and Video

In the code snippet below, we import the required dependencies for our project, initialize a YOLOv8 model, and load a video into our project.

In [14]:
import numpy as np
import supervision as sv
import cv2

from ultralytics import YOLO

model = YOLO("yolov8s.pt")

VIDEO = "video.mp4"

# Use the correct ColorPalette initialization
colors = sv.ColorPalette.DEFAULT

# Check if video file exists, if not we'll handle it later
try:
    video_info = sv.VideoInfo.from_video_path(VIDEO)
    print(f"Video info: {video_info}")
except FileNotFoundError:
    print(f"Video file '{VIDEO}' not found. Please download it first.")
    video_info = None

Video info: VideoInfo(width=1920, height=1080, fps=29, total_frames=1275)


## Save Frame

The code snippet below saves the first frame in your video to a file called "first_frame.png".

In [16]:
# extract video frame
generator = sv.get_video_frames_generator(VIDEO)
iterator = iter(generator)

frame = next(iterator)

# save first frame
cv2.imwrite("first_frame.png", frame)

True

Next, go to [PolygonZone](https://roboflow.github.io/polygonzone/) and draw polygons on your image. PolygonZone returns a list of polygon coordinates in both NumPy and JSON formats. Copy the NumPy output into the cell below:

In [17]:
polygons = [
    np.array([[718, 595], [927, 592], [851, 1062], [42, 1059]]),
    np.array([[987, 595], [1199, 595], [1893, 1056], [1015, 1062]]),
]

## Run Inference

Using the YOLOv8 model we initialized earlier, as well as our Supervision objects, we can draw polygons on the first frame on our image and count the number of objects that appear.

First, let's initialize our zones:

In [24]:
# initialize our zones

zones = [
    sv.PolygonZone(polygon=polygon)  # Remove frame_resolution_wh parameter
    for polygon in polygons
]
zone_annotators = [
    sv.PolygonZoneAnnotator(
        zone=zone,
        color=colors.by_idx(index),
        thickness=4,
        text_thickness=8,
        text_scale=4,
    )
    for index, zone in enumerate(zones)
]
box_annotators = [
    sv.BoxAnnotator(
        color=colors.by_idx(index), 
        thickness=4
    )
    for index in range(len(polygons))
]


def process_frame(frame: np.ndarray, i) -> np.ndarray:
    results = model(frame, imgsz=1280, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    for zone, zone_annotator, box_annotator in zip(
        zones, zone_annotators, box_annotators
    ):
        mask = zone.trigger(detections=detections)
        detections_filtered = detections[mask]
        frame = box_annotator.annotate(
            scene=frame, detections=detections_filtered
        )
        frame = zone_annotator.annotate(scene=frame)

    return frame

Now we can run inference. Let's run inference on a single frame so we can make sure everything is working as expected:

In [22]:
results = model(frame, imgsz=1280, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results)

for zone, zone_annotator, box_annotator in zip(zones, zone_annotators, box_annotators):
    mask = zone.trigger(detections=detections)
    detections_filtered = detections[mask]
    frame = box_annotator.annotate(scene=frame, detections=detections_filtered)
    frame = zone_annotator.annotate(scene=frame)

# Display the frame with matplotlib backend fix
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 16))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Polygon Zone Detection Results')
plt.tight_layout()
plt.show()

The frame above shows all of the predictions in the polygons we have drawn. Now we can proceed to run inference on the rest of the video.

## Video Inference

Use the code snippet below to run inference on the video you specified earlier and save the results to "result.mp4".

In [25]:
sv.process_video(source_path=VIDEO, target_path="result.mp4", callback=process_frame)